Librerias

In [ ]:
import pandas as pd
import numpy as np
import os
import torch
import math
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

Importacion de datos

In [13]:
carpeta = '/home/afpuertav/mySpace/archivos/lotesSentido'
csv_files = [f for f in os.listdir(carpeta) if f.endswith('.csv')]
df = pd.concat([pd.read_csv(os.path.join(carpeta, f)) for f in csv_files], ignore_index=True)

Tratamiento de valencia

In [14]:
df

,Author,Content,Date,Language,Location,Number of Likes,Number of Retweets,In Reply To,Author Name,Author Description,...,Author Verified,Longitude,Latitude,Sentimiento,Emocion,Content_limpio,nmf_topic_id,nmf_topic_keywords,cantidad de palabras,perplejidad
0,@quepasaraucania,[COVID-19 EN LA ARAUCANÍA 🦠] La Araucanía aman...,2020-06-03 18:56:49,es,NaN,0.0,0.0,NaN,Qué pasa Araucanía,"Medio independiente, sin tendencia política ni...",...,False,NaN,NaN,1 star,caring,[covid-19 EN LA ARAUCANIA ] La Araucania amane...,7,salud ministerio personal pública sistema,27.0,292.404959
1,@Digital_58,Alarma en el deporte japones tras confirmarse ...,2020-06-03 18:56:47,es,NaN,0.0,0.0,NaN,Noticias Digital58 🇻🇪,#PeriodismoWebDeVerdad\n✉ prensa@digital58.com...,...,False,NaN,NaN,1 star,nervousness,Alarma en el deporte japones tras confirmarse ...,1,casos nuevos positivos confirmados fallecidos,11.0,677.033785
2,@SIPSENOTICIAS,"#SNCUN Hasta las 12 horas del 03 de junio, se ...",2020-06-03 18:56:45,es,NaN,0.0,0.0,NaN,SIPSE NOTICIAS,Grupo SIPSE empresa líder en información en el...,...,False,NaN,NaN,1 star,caring,"#SNCUN Hasta las 12 horas del 03 de junio, se ...",11,personas fallecidas total recuperadas contagiadas,29.0,271.657029
3,@Olivareslucia,"#LaPreguntaExacta ¿Qué temas, además de la pan...",2020-06-03 18:56:44,es,NaN,0.0,0.0,NaN,Lucía Olivares,Comunicadora. Autora de #SeñoraBruja. Me escuc...,...,False,NaN,NaN,1 star,approval,"#LaPreguntaExacta Que temas, ademas de la pand...",8,méxico muertos defunciones confirmados ssaludmx,18.0,503.123030
4,@seleneverar_,* Tania Tinoco a Xavier Vallarino: ¿puede deci...,2020-06-03 18:56:41,es,EC,0.0,0.0,NaN,Selene Vera R.,#Periodista y #Dircom •Sin fines de semana •En...,...,False,NaN,NaN,1 star,caring,* Tania Tinoco a Xavier Vallarino: puede decir...,4,pandemia mundo medio tiempos mundial,40.0,515.409013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1325105,@MunicipioEE,Informe N° 265. 15 detenidos en controles en v...,2020-05-09 13:15:32,es,NaN,1.0,0.0,NaN,Esteban Echeverría,Cuenta oficial del Municipio de Esteban Echeve...,...,False,NaN,NaN,1 star,caring,Informe N 265. 15 detenidos en controles en vi...,12,vía youtube españa eluniversalmx cuba,13.0,664.351412
1325106,@bbva,¿Puede transmitirse la #COVID19 a través del a...,2020-05-09 13:15:30,es,NaN,10.0,2.0,NaN,BBVA,"Cuestionamos lo establecido, buscamos nuevos c...",...,True,NaN,NaN,1 star,caring,Puede transmitirse la #covid-19 a traves del a...,3,gobierno crisis cuarentena ser ahora,25.0,380.457434
1325107,@alexlibre18,"@HLGatell Ok Hugo, te lo compro. \n\nAhora bie...",2020-05-09 13:15:25,es,NaN,0.0,0.0,@HLGatell,LEX,libertad de expresión,...,False,NaN,NaN,5 stars,approval,"Ok Hugo, te lo compro. Ahora bien, que % de lo...",3,gobierno crisis cuarentena ser ahora,29.0,311.852359
1325108,@30_marval,Evita reuniones y fiestas durante la emergenci...,2020-05-09 13:15:21,es,NaN,0.0,0.0,NaN,El profe marval,"NACI EN GDL,REGISTRADO EN LA DELEGACION DE SAN...",...,False,NaN,NaN,1 star,caring,Evita reuniones y fiestas durante la emergenci...,11,personas fallecidas total recuperadas contagiadas,38.0,163.165993


In [15]:
df = df.drop_duplicates(subset='Content_limpio', keep='first').reset_index(drop=True)

In [19]:
mapa = {'1 star' : 'Negative',
        '2 stars' : 'Negative',
        '3 stars' : 'Neutral',
        '4 stars': 'Postive',
        '5 stars' : 'Positive',
    }

In [20]:
df['Sentimiento'].replace(mapa)

0          Negative
1          Negative
2          Negative
3          Negative
4          Negative
             ...   
1219016    Negative
1219017    Negative
1219018    Positive
1219019    Negative
1219020    Negative
Name: Sentimiento, Length: 1219021, dtype: object

In [21]:
df

,Author,Content,Date,Language,Location,Number of Likes,Number of Retweets,In Reply To,Author Name,Author Description,...,Author Verified,Longitude,Latitude,Sentimiento,Emocion,Content_limpio,nmf_topic_id,nmf_topic_keywords,cantidad de palabras,perplejidad
0,@quepasaraucania,[COVID-19 EN LA ARAUCANÍA 🦠] La Araucanía aman...,2020-06-03 18:56:49,es,NaN,0.0,0.0,NaN,Qué pasa Araucanía,"Medio independiente, sin tendencia política ni...",...,False,NaN,NaN,1 star,caring,[covid-19 EN LA ARAUCANIA ] La Araucania amane...,7,salud ministerio personal pública sistema,27.0,292.404959
1,@Digital_58,Alarma en el deporte japones tras confirmarse ...,2020-06-03 18:56:47,es,NaN,0.0,0.0,NaN,Noticias Digital58 🇻🇪,#PeriodismoWebDeVerdad\n✉ prensa@digital58.com...,...,False,NaN,NaN,1 star,nervousness,Alarma en el deporte japones tras confirmarse ...,1,casos nuevos positivos confirmados fallecidos,11.0,677.033785
2,@SIPSENOTICIAS,"#SNCUN Hasta las 12 horas del 03 de junio, se ...",2020-06-03 18:56:45,es,NaN,0.0,0.0,NaN,SIPSE NOTICIAS,Grupo SIPSE empresa líder en información en el...,...,False,NaN,NaN,1 star,caring,"#SNCUN Hasta las 12 horas del 03 de junio, se ...",11,personas fallecidas total recuperadas contagiadas,29.0,271.657029
3,@Olivareslucia,"#LaPreguntaExacta ¿Qué temas, además de la pan...",2020-06-03 18:56:44,es,NaN,0.0,0.0,NaN,Lucía Olivares,Comunicadora. Autora de #SeñoraBruja. Me escuc...,...,False,NaN,NaN,1 star,approval,"#LaPreguntaExacta Que temas, ademas de la pand...",8,méxico muertos defunciones confirmados ssaludmx,18.0,503.123030
4,@seleneverar_,* Tania Tinoco a Xavier Vallarino: ¿puede deci...,2020-06-03 18:56:41,es,EC,0.0,0.0,NaN,Selene Vera R.,#Periodista y #Dircom •Sin fines de semana •En...,...,False,NaN,NaN,1 star,caring,* Tania Tinoco a Xavier Vallarino: puede decir...,4,pandemia mundo medio tiempos mundial,40.0,515.409013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219016,@MunicipioEE,Informe N° 265. 15 detenidos en controles en v...,2020-05-09 13:15:32,es,NaN,1.0,0.0,NaN,Esteban Echeverría,Cuenta oficial del Municipio de Esteban Echeve...,...,False,NaN,NaN,1 star,caring,Informe N 265. 15 detenidos en controles en vi...,12,vía youtube españa eluniversalmx cuba,13.0,664.351412
1219017,@bbva,¿Puede transmitirse la #COVID19 a través del a...,2020-05-09 13:15:30,es,NaN,10.0,2.0,NaN,BBVA,"Cuestionamos lo establecido, buscamos nuevos c...",...,True,NaN,NaN,1 star,caring,Puede transmitirse la #covid-19 a traves del a...,3,gobierno crisis cuarentena ser ahora,25.0,380.457434
1219018,@alexlibre18,"@HLGatell Ok Hugo, te lo compro. \n\nAhora bie...",2020-05-09 13:15:25,es,NaN,0.0,0.0,@HLGatell,LEX,libertad de expresión,...,False,NaN,NaN,5 stars,approval,"Ok Hugo, te lo compro. Ahora bien, que % de lo...",3,gobierno crisis cuarentena ser ahora,29.0,311.852359
1219019,@30_marval,Evita reuniones y fiestas durante la emergenci...,2020-05-09 13:15:21,es,NaN,0.0,0.0,NaN,El profe marval,"NACI EN GDL,REGISTRADO EN LA DELEGACION DE SAN...",...,False,NaN,NaN,1 star,caring,Evita reuniones y fiestas durante la emergenci...,11,personas fallecidas total recuperadas contagiadas,38.0,163.165993


In [22]:
df.to_csv('/home/afpuertav/mySpace/archivos/tweetsCompletos20250616.csv', index=False)